# Processamento Automatizado de Documentos Técnicos – Projeto IAVS
Este notebook reúne todos os scripts utilizados no processo de extração, limpeza e organização dos documentos técnicos normativos no âmbito do projeto IAVS (Inteligência Artificial para Vigilância em Saúde).

O objetivo geral é estruturar uma base textual confiável e padronizada que possa ser utilizada em processos de classificação, análise e apoio a modelos de inteligência artificial.

Etapas contempladas:

- Preparação do Ambiente

- Importação dos pacotes necessários.

- Definição dos caminhos e criação automática das pastas que compõem o fluxo de trabalho.

- Funções Utilitárias

- Função de normalização de texto para padronizar nomes de arquivos.

- Função para localização automatizada de arquivos PDF por similaridade textual (fuzzy matching).

- Função de limpeza textual para corrigir formatações e eliminar ruídos típicos de extração.

- Classificação e Renomeação

- Leitura da planilha de controle (controle_documentos.xlsx).

- Localização, categorização e cópia dos arquivos PDF com nomes padronizados por doença e tipo de documento.

- Conversão de PDFs para Texto

- Extração do conteúdo textual dos documentos PDF.

- Geração de arquivos .txt para cada documento localizado.

- Limpeza dos Arquivos de Texto

- Remoção de quebras de linha indesejadas, espaços repetidos e artefatos como marcações de página.

- Verificação dos Arquivos Gerados

- Listagem final dos arquivos tratados e prontos para uso, como etapa de validação visual.

**Observações:**
Todas as operações são feitas de forma automatizada, com verificações para evitar retrabalho e perdas de informação.

O pipeline é compatível com bases documentais de grande volume e pode ser reaplicado para outras áreas temáticas da vigilância em saúde.

## Etapa 1 – Definição de Estrutura e Preparação do Ambiente
Este trecho do código realiza a preparação inicial do ambiente de trabalho para o processamento automatizado de documentos no projeto IAVS (Inteligência Artificial para Vigilância em Saúde).

**Objetivos:**
- Importar os módulos necessários para manipulação de arquivos, textos, planilhas e PDFs;

- Definir o diretório base do projeto e as subpastas que organizam os arquivos em diferentes estágios do fluxo (originais, convertidos, limpos e organizados);

- Criar automaticamente as pastas exigidas pelo pipeline, garantindo que a estrutura esteja pronta mesmo em execuções futuras ou em novos ambientes.

Essa etapa garante uma organização padronizada dos arquivos e prepara o caminho para a execução segura e escalável dos scripts de extração, limpeza e categorização dos documentos normativos da área da saúde.



In [1]:
# Importação dos módulos necessários
%pip install pdfplumber pandas openpyxl
# Importando bibliotecas essenciais
import os   # Operações com diretórios e caminhos de arquivos
import pandas as pd        # Leitura e manipulação de planilhas (DataFrames)
import re # Expressões regulares para manipulação de texto
import hashlib
import csv
from pathlib import Path
import pdfplumber          # Extração de texto de arquivos PDF
import time  # Módulo para medir tempo de execução
import shutil      # Operações de cópia e movimentação de arquivos
import unicodedata # Manipulação de acentuação e caracteres Unicode
import difflib     # Comparação de similaridade entre strings
from collections import Counter  # Contagem de elementos (como palavras ou erros)

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Definição do diretório base do projeto
BASE_DIR = r"C:\Users\isisi\Documents\IAVS_PROJETO"

# Caminho da planilha de controle dos documentos
PLANILHA = os.path.join(BASE_DIR, "controle_documentos.xlsx")

# Definição das pastas utilizadas no fluxo de trabalho
PASTA_BRUTOS = os.path.join(BASE_DIR, "pdfs_brutos")           # PDF originais, sem tratamento
PASTA_CONVERTIDOS = os.path.join(BASE_DIR, "textos_convertidos")  # Arquivos .txt gerados a partir dos PDFs
PASTA_LIMPOS = os.path.join(BASE_DIR, "textos_limpos")         # Textos tratados e limpos (sem cabeçalho, rodapé etc.)
PASTA_FINAL = os.path.join(BASE_DIR, "organizados")            # Textos organizados por doença e tema

# Criação automática das pastas, se ainda não existirem
for pasta in [PASTA_BRUTOS, PASTA_CONVERTIDOS, PASTA_LIMPOS, PASTA_FINAL]:
    os.makedirs(pasta, exist_ok=True)

## Etapa 2 – Funções de Normalização, Localização e Limpeza de Texto
Esta etapa define funções auxiliares fundamentais para garantir a padronização dos nomes de arquivos, a localização dos PDFs corretos e a limpeza dos textos extraídos.

Funções implementadas:
- normalizar_texto(texto)
- Padroniza o nome de documentos ou qualquer outro texto, tornando-o uniforme para fins de comparação e nomenclatura. Remove acentos, converte para minúsculas e substitui - caracteres especiais por underline (_). Essa função é essencial para que os nomes de arquivos e títulos da planilha sejam comparáveis de forma robusta.

- localizar_pdf_local(nome_documento_original)
Busca, dentro das subpastas do projeto, o arquivo PDF correspondente ao nome fornecido na planilha. Para isso, utiliza a função normalizar_texto e a técnica de fuzzy matching (semelhança textual aproximada) via difflib.get_close_matches, permitindo identificar arquivos mesmo que existam pequenas variações no nome.

- limpar_texto(texto)

- Remove ruídos comuns nos textos extraídos de PDFs, como cabeçalhos, rodapés e quebras de linha desnecessárias. 

A função:

- Elimina linhas muito repetidas no corpo do texto;

- Junta quebras de linha isoladas (geralmente causadas por limites de página);

- Mantém as separações entre parágrafos com duplas quebras de linha.

*Estas funções são reutilizadas em diversas etapas do pipeline e garantem a qualidade e a organização dos dados processados para uso posterior em análises, indexação e treinamento da IA.*

In [3]:
# === CONFIGURAÇÃO DO CAMINHO DO ARQUIVO DE CONTROLE ===
CAMINHO_PLANILHA = r"C:\Users\isisi\Documents\IAVS_PROJETO\controle_documentos.xlsx"
NOME_ABA = "docs"
LOG_DIAGNOSTICO = r"C:\Users\isisi\Documents\IAVS_PROJETO\log_diagnostico_etapa2.csv"

# === FUNÇÕES AUXILIARES ===

def calcular_hash(caminho_arquivo, limite=1024*1024):
    """Calcula hash de uma parte do arquivo (1 MB por padrão) para diagnóstico rápido."""
    sha256 = hashlib.sha256()
    with open(caminho_arquivo, 'rb') as f:
        sha256.update(f.read(limite))
    return sha256.hexdigest()

def normalizar_nome(nome):
    """Remove repetições consecutivas de sufixos no nome."""
    padrao = r'(txt_\w+\d{4})+'
    return re.sub(padrao, lambda m: m.group(1), nome)

def diagnosticar_planilha(planilha_path, aba, log_path):
    df = pd.read_excel(planilha_path, sheet_name=aba)
    hash_map = {}
    log_linhas = []

    # 🔄 Carrega os caminhos já processados anteriormente (se houver log salvo)
    arquivos_ja_processados = set()
    if os.path.exists(log_path):
        log_antigo = pd.read_csv(log_path)
        arquivos_ja_processados = set(log_antigo["Arquivo/Caminho"].dropna().unique())

    # 🔁 Loop sobre cada linha da planilha
    for idx, row in df.iterrows():
        caminho = str(row.get("Caminho", "")).strip()
        nome = str(row.get("Nome do arquivo", "")).strip()

        # ⏭️ Pula arquivos já processados antes
        if caminho in arquivos_ja_processados:
            continue

        if not caminho or not nome:
            log_linhas.append(["ERRO", f"Linha {idx+2}", "Caminho ou Nome do arquivo vazio", ""])
            continue

        if not os.path.isfile(caminho):
            log_linhas.append(["NÃO ENCONTRADO", caminho, "Arquivo não localizado", ""])
            continue

        # 🔎 Diagnóstico de duplicidade por hash
        hash_arquivo = calcular_hash(caminho)
        if hash_arquivo in hash_map:
            log_linhas.append(["DUPLICADO", caminho, "Hash igual a", hash_map[hash_arquivo]])
        else:
            hash_map[hash_arquivo] = caminho

        # 🧹 Diagnóstico de nome corrompido
        nome_base = os.path.basename(caminho).replace(".pdf", "")
        nome_normalizado = normalizar_nome(nome_base)
        if nome != nome_normalizado:
            log_linhas.append(["NOME_CORROMPIDO", caminho, "Nome informado difere do sugerido", nome_normalizado])

    # 💾 Salva ou atualiza o log
    if os.path.exists(log_path) and log_linhas:
        log_df_antigo = pd.read_csv(log_path)
        log_df_novo = pd.DataFrame(log_linhas, columns=["Tipo", "Arquivo/Caminho", "Info", "Referência/Sugestão"])
        log_df_final = pd.concat([log_df_antigo, log_df_novo], ignore_index=True)
        log_df_final.to_csv(log_path, index=False, encoding='utf-8')
        print(f"✅ Diagnóstico concluído. {len(log_linhas)} novos registros adicionados ao log.")
        print(f"📄 Log atualizado em: {log_path}")

    elif log_linhas:
        with open(log_path, mode='w', newline='', encoding='utf-8') as log_csv:
            writer = csv.writer(log_csv)
            writer.writerow(["Tipo", "Arquivo/Caminho", "Info", "Referência/Sugestão"])
            writer.writerows(log_linhas)
        print(f"✅ Diagnóstico concluído. {len(log_linhas)} registros salvos no log novo.")
        print(f"📄 Log salvo em: {log_path}")
    
    else:
        print("ℹ️ Nenhum novo arquivo a processar. Log mantido inalterado.")

# === EXECUTAR ===
if __name__ == "__main__":
    diagnosticar_planilha(CAMINHO_PLANILHA, NOME_ABA, LOG_DIAGNOSTICO)


ℹ️ Nenhum novo arquivo a processar. Log mantido inalterado.


##  Etapa 3 – Classificação e Renomeação dos Documentos por Doença e Tipo
Este trecho do script realiza a leitura da planilha de controle (controle_documentos.xlsx) e organiza os documentos PDF brutos em uma estrutura padronizada baseada nas doenças abordadas e no tipo de documento.

**Objetivos:**
- Ler os metadados dos documentos a partir da planilha preenchida manualmente;

- Ignorar registros incompletos ou genéricos (como documentos marcados com “Diversas doenças”);

- Utilizar a função localizar_pdf_local() para encontrar, via fuzzy matching, o caminho correto do PDF nas subpastas do projeto;

- Para cada doença mencionada no documento:

- Criar uma chave única combinando o nome da doença com o tipo de documento (ex: tuberculose_protocolo);

- Utilizar um contador para evitar sobrescrições (ex: tuberculose_protocolo_1.pdf, tuberculose_protocolo_2.pdf...);

- Copiar o arquivo PDF correspondente para a pasta pdfs_brutos, já com o novo nome padronizado.

**Resultado esperado:**
Ao final da execução, a pasta pdfs_brutos conterá todos os documentos localizados e renomeados de forma padronizada e rastreável, facilitando os próximos passos de extração de texto, organização temática e posterior uso pela IA.

In [4]:
# === CONFIGURAÇÕES ===
PLANILHA = r"C:\Users\isisi\Documents\IAVS_PROJETO\controle_documentos.xlsx"
NOME_ABA = "docs"
PASTA_BRUTOS = r"C:\Users\isisi\Documents\IAVS_PROJETO\pdfs_brutos"

# Garante que a pasta de destino existe
os.makedirs(PASTA_BRUTOS, exist_ok=True)

# Carrega a planilha de controle com os metadados dos documentos
df = pd.read_excel(PLANILHA, sheet_name=NOME_ABA)

# Dicionário para contar quantas vezes uma combinação (doença + tipo) já foi usada no nome de arquivos
contador_map = {}

# Itera sobre cada linha da planilha
for _, row in df.iterrows():
    nome_doc = str(row.get("Nome do Documento", "")).strip()
    tipo = str(row.get("Tipo", "")).lower().replace(" ", "_").strip()
    doencas_raw = str(row.get("Doenças abordadas", "")).strip()
    caminho_pdf = str(row.get("Caminho", "")).strip()

    # Ignora entradas incompletas ou genéricas
    if not nome_doc or not tipo or not doencas_raw or not caminho_pdf:
        continue
    if doencas_raw.lower().startswith("diversas") or not os.path.isfile(caminho_pdf):
        continue

    # Separa múltiplas doenças por delimitadores comuns
    doencas_lista = [d.strip().lower().replace(" ", "_") for d in re.split(r'[;,/]', doencas_raw)]

    for doenca in doencas_lista:
        chave = (doenca, tipo)
        contador_map[chave] = contador_map.get(chave, 0) + 1
        novo_nome = f"{doenca}_{tipo}_{contador_map[chave]}.pdf"
        destino = os.path.join(PASTA_BRUTOS, novo_nome)

        if os.path.exists(destino):
            print(f"⏭️ Já existente, ignorado: {novo_nome}")
            continue

        try:
            shutil.copy(caminho_pdf, destino)
            print(f"✅ Copiado: {novo_nome}")
        except Exception as e:
            print(f"❌ Erro ao copiar {caminho_pdf} → {destino}: {e}")


⏭️ Já existente, ignorado: dengue_nota_técnica_1.pdf
⏭️ Já existente, ignorado: chikungunya_nota_técnica_1.pdf
⏭️ Já existente, ignorado: zika_nota_técnica_1.pdf
⏭️ Já existente, ignorado: febre_amarela_nota_técnica_1.pdf
⏭️ Já existente, ignorado: febre_do_nilo_ocidental_nota_técnica_1.pdf
⏭️ Já existente, ignorado: arboviroses_nota_técnica_1.pdf
⏭️ Já existente, ignorado: covid-19_nota_técnica_1.pdf
⏭️ Já existente, ignorado: dengue_protocolo_1.pdf
⏭️ Já existente, ignorado: sarampo_nota_técnica_1.pdf
⏭️ Já existente, ignorado: rubéola_nota_técnica_1.pdf
⏭️ Já existente, ignorado: coqueluche_nota_técnica_1.pdf
⏭️ Já existente, ignorado: difteria_nota_técnica_1.pdf
⏭️ Já existente, ignorado: tétano_nota_técnica_1.pdf
⏭️ Já existente, ignorado: hepatites_nota_técnica_1.pdf
⏭️ Já existente, ignorado: varicela_nota_técnica_1.pdf
⏭️ Já existente, ignorado: entre_outras_doenças_imunopreveníveis_nota_técnica_1.pdf
⏭️ Já existente, ignorado: dengue_protocolo_2.pdf
⏭️ Já existente, ignorado: 

**Etapa 4 – Conversão de PDFs para Texto (.txt)**
Este bloco realiza a conversão dos arquivos PDF classificados na etapa anterior para arquivos de texto simples (.txt), permitindo que seu conteúdo possa ser tratado, limpo e analisado posteriormente.

Objetivos:
- Ler os PDFs localizados na pasta pdfs_brutos;

- Verificar se a conversão já foi feita anteriormente para evitar retrabalho;

- Utilizar a biblioteca pdfplumber para extrair o texto de cada página;

- Salvar o conteúdo textual extraído em arquivos .txt com codificação UTF-8;

- Exibir mensagens de progresso, avisos de falhas e medir o tempo total da operação.

*Essa etapa é fundamental para garantir que todo o conteúdo dos documentos esteja disponível em formato editável, possibilitando sua limpeza e categorização por inteligência artificial.*

In [5]:
# === CONFIGURAÇÕES ===
PASTA_BRUTOS = r"C:\Users\isisi\Documents\IAVS_PROJETO\pdfs_brutos"
PASTA_CONVERTIDOS = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_convertidos"

# Garante que a pasta de destino existe
os.makedirs(PASTA_CONVERTIDOS, exist_ok=True)

# ⏱️ Início da contagem de tempo
start = time.time()

# Itera sobre todos os arquivos da pasta de PDFs brutos
for arquivo in os.listdir(PASTA_BRUTOS):
    if arquivo.endswith(".pdf"):
        # Define o nome e caminho do arquivo .txt correspondente
        nome_txt = arquivo.replace(".pdf", ".txt")
        caminho_txt = os.path.join(PASTA_CONVERTIDOS, nome_txt)

        # ✅ Pula se o arquivo já foi convertido anteriormente
        if os.path.exists(caminho_txt):
            print(f"⏭️ Já convertido: {nome_txt}")
            continue

        # Caminho completo do PDF de origem
        caminho_pdf = os.path.join(PASTA_BRUTOS, arquivo)
        print(f"🔄 Convertendo: {arquivo}")

        try:
            # Abre o PDF com pdfplumber e extrai o texto de cada página
            with pdfplumber.open(caminho_pdf) as pdf:
                texto = "\n".join(page.extract_text() or '' for page in pdf.pages)

            # Verifica se o texto extraído está vazio
            if not texto.strip():
                print(f"⚠️ Texto vazio: {arquivo}")
                continue

            # Salva o texto extraído em um arquivo .txt
            with open(caminho_txt, "w", encoding="utf-8") as f:
                f.write(texto)

            print(f"✅ Texto salvo: {nome_txt}")

        except Exception as e:
            # Exibe erro caso a conversão falhe
            print(f"❌ Erro ao converter {arquivo}: {e}")

# ⏱️ Exibe tempo total de execução da conversão
print(f"⏱️ Conversão finalizada em {time.time() - start:.2f} segundos")


⏭️ Já convertido: adenovírus_guia_1.txt
⏭️ Já convertido: arboviroses_nota_técnica_1.txt
⏭️ Já convertido: cancro_mole_protocolo_1.txt
⏭️ Já convertido: chikungunya_nota_técnica_1.txt
⏭️ Já convertido: chikungunya_protocolo_1.txt
⏭️ Já convertido: chikungunya_protocolo_2.txt
⏭️ Já convertido: chikungunya_protocolo_3.txt
⏭️ Já convertido: clamídia_protocolo_1.txt
⏭️ Já convertido: coinfecção_com_hepatite_b_nota_técnica_1.txt
⏭️ Já convertido: coqueluche_nota_técnica_1.txt
⏭️ Já convertido: coqueluche_nota_técnica_2.txt
⏭️ Já convertido: coqueluche_nota_técnica_3.txt
⏭️ Já convertido: covid-19_guia_1.txt
⏭️ Já convertido: covid-19_nota_técnica_1.txt
⏭️ Já convertido: cólera_nota_técnica_1.txt
⏭️ Já convertido: dengue_nota_técnica_1.txt
⏭️ Já convertido: dengue_protocolo_1.txt
⏭️ Já convertido: dengue_protocolo_2.txt
⏭️ Já convertido: dengue_protocolo_3.txt
⏭️ Já convertido: dengue_protocolo_4.txt
⏭️ Já convertido: difteria_nota_técnica_1.txt
⏭️ Já convertido: entre_outras_doenças_imunopr

## Etapa 5 – Limpeza dos Textos Extraídos
Após a conversão dos PDFs em arquivos .txt, esta etapa realiza uma limpeza estruturada do conteúdo textual para remover ruídos típicos da extração, como espaços excessivos, quebras de linha desnecessárias, artefatos de paginação e formatações indesejadas.

**Objetivos:**
- Processar todos os arquivos .txt gerados anteriormente;

- Aplicar uma função de limpeza que corrige:

- Quebras de linha duplas ou múltiplas;

- Espaços repetidos e tabulações;

- Espaços no início de linhas;

- Padrões comuns de numeração de páginas (“Página X de Y”);

- Salvar os textos tratados na pasta textos_limpos, preservando o nome do arquivo original.

*Essa limpeza é essencial para garantir que o conteúdo esteja formatado de maneira uniforme e pronto para ser classificado, indexado e posteriormente analisado por modelos de inteligência artificial.*

In [6]:
# === CONFIGURAÇÕES ===
PASTA_CONVERTIDOS = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_convertidos"
PASTA_LIMPOS = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_limpos"
LOG_DIAGNOSTICO = os.path.join(PASTA_LIMPOS, "log_diagnostico.csv")
os.makedirs(PASTA_LIMPOS, exist_ok=True)

def limpar_texto(texto):
    """
    Realiza limpeza básica do texto extraído:
    - Remove múltiplas quebras de linha
    - Substitui espaços e tabulações repetidas por um único espaço
    - Remove espaços no início das linhas
    - Elimina marcações de página como "Página 3 de 12"
    """
    texto = re.sub(r'\n{2,}', '\n\n', texto)
    texto = re.sub(r'[ \t]+', ' ', texto)
    texto = re.sub(r'^\s+', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'Página\s+\d+\s+de\s+\d+', '', texto, flags=re.IGNORECASE)
    return texto.strip()

def calcular_hash(caminho_arquivo, limite=1024*1024):
    """Calcula hash parcial (1MB) de um arquivo."""
    sha256 = hashlib.sha256()
    with open(caminho_arquivo, 'rb') as f:
        sha256.update(f.read(limite))
    return sha256.hexdigest()

# === CARREGA HASHES JÁ PROCESSADOS DO LOG ===
hash_map = {}
if os.path.exists(LOG_DIAGNOSTICO):
    with open(LOG_DIAGNOSTICO, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # pula o cabeçalho
        for linha in reader:
            if linha[0] == "DUPLICADO":
                continue
            hash_map[linha[1]] = linha[3]  # caminho → hash salvo

# === PROCESSA ARQUIVOS .TXT ===
log_linhas = []

print(f"Limpando arquivos em: {PASTA_CONVERTIDOS}")
for arquivo in os.listdir(PASTA_CONVERTIDOS):
    if arquivo.endswith(".txt"):
        caminho_origem = os.path.join(PASTA_CONVERTIDOS, arquivo)
        caminho_destino = os.path.join(PASTA_LIMPOS, arquivo)

        hash_origem = calcular_hash(caminho_origem)

        # Se já processado e hash bate, pula
        if caminho_origem in hash_map and hash_map[caminho_origem] == hash_origem:
            print(f"⏭️ Já limpo: {arquivo}")
            continue

        try:
            with open(caminho_origem, "r", encoding="utf-8", errors="ignore") as f:
                conteudo = f.read()

            texto_limpo = limpar_texto(conteudo)

            with open(caminho_destino, "w", encoding="utf-8") as f:
                f.write(texto_limpo)

            print(f"✅ Limpo e salvo: {arquivo}")
            log_linhas.append(["LIMPO", caminho_origem, "Hash original", hash_origem])

        except Exception as e:
            print(f"❌ Erro ao processar {arquivo}: {e}")
            log_linhas.append(["ERRO", caminho_origem, "Exceção", str(e)])

# === SALVA LOG FINAL ===
with open(LOG_DIAGNOSTICO, mode='a', newline='', encoding='utf-8') as log_csv:
    writer = csv.writer(log_csv)
    if os.stat(LOG_DIAGNOSTICO).st_size == 0:
        writer.writerow(["Tipo", "Arquivo", "Info", "Referência/Sugestão"])
    writer.writerows(log_linhas)

print("🏁 Finalizado.")


Limpando arquivos em: C:\Users\isisi\Documents\IAVS_PROJETO\textos_convertidos
⏭️ Já limpo: adenovírus_guia_1.txt
⏭️ Já limpo: arboviroses_nota_técnica_1.txt
⏭️ Já limpo: cancro_mole_protocolo_1.txt
⏭️ Já limpo: chikungunya_nota_técnica_1.txt
⏭️ Já limpo: chikungunya_protocolo_1.txt
⏭️ Já limpo: chikungunya_protocolo_2.txt
⏭️ Já limpo: chikungunya_protocolo_3.txt
⏭️ Já limpo: clamídia_protocolo_1.txt
⏭️ Já limpo: coinfecção_com_hepatite_b_nota_técnica_1.txt
⏭️ Já limpo: coqueluche_nota_técnica_1.txt
⏭️ Já limpo: coqueluche_nota_técnica_2.txt
⏭️ Já limpo: coqueluche_nota_técnica_3.txt
⏭️ Já limpo: covid-19_guia_1.txt
⏭️ Já limpo: covid-19_nota_técnica_1.txt
⏭️ Já limpo: cólera_nota_técnica_1.txt
⏭️ Já limpo: dengue_nota_técnica_1.txt
⏭️ Já limpo: dengue_protocolo_1.txt
⏭️ Já limpo: dengue_protocolo_2.txt
⏭️ Já limpo: dengue_protocolo_3.txt
⏭️ Já limpo: dengue_protocolo_4.txt
⏭️ Já limpo: difteria_nota_técnica_1.txt
⏭️ Já limpo: entre_outras_doenças_imunopreveníveis_nota_técnica_1.txt
⏭️

In [7]:
# === CONFIGURAÇÕES ===
PASTA_ORIGEM = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_convertidos"
PASTA_DESTINO = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_limpos"
os.makedirs(PASTA_DESTINO, exist_ok=True)

# === PADRÕES E SÍMBOLOS A REMOVER ===
CARACTERES_CORROMPIDOS = ['', '', '']

PADROES_LINHAS_REMOVER = [
    r'^Página\s+\d+\s+de\s+\d+',
    r'^ISBN\s+',
    r'^202\d+.*',
    r'^\d{1,3}$',
    r'^[A-ZÀ-Ú\s]{2,}$',
    r'^http[s]?://',
    r'^Ministério da Saúde',
    r'^Coordenação-Geral|^Departamento|^Secretaria|^Editora|^Projeto gráfico',
    r'^Ficha Catalográfica',
    r'^Catalogação na fonte',
    r'^Lista de abreviaturas e siglas$',
    r'^Sumário$',
    r'^.*TABELA\s+\d+.*$',
    r'^[^\w\s]*$',
    r'^.{5,}\.{3,}\s*\d{1,3}$',
]

PADROES_TRECHOS_REMOVER = [
    r'Ministério da Saúde.*?www\.saude\.gov\.br.*?\n\n',
    r'©? ?202\d+.*?(Creative Commons|Licença|Reprodução|Distribuição).*?\n\n',
    r'Projeto gráfico:.*?Revisão:.*?\n\n',
]

# === FUNÇÕES ===

def remover_blocos_de_sumario(texto): #remover blocos de sumário
    """
    Remove linhas que parecem ser sumários, mantendo apenas as linhas recentes.
    Um sumário é identificado por um padrão específico de numeração e texto.
    """
    linhas = texto.splitlines()
    linhas_filtradas = []
    padrao_sumario = re.compile(r'^\d+(\.\d+)*\s+.+?\.{3,}\s*\d{1,3}$')
    linhas_recentes = []

    for linha in linhas:
        linha_strip = linha.strip()
        if padrao_sumario.match(linha_strip):
            linhas_recentes.append(linha_strip)
            continue
        elif linhas_recentes:
            linhas_recentes = []
        linhas_filtradas.append(linha)

    return "\n".join(linhas_filtradas)

def remover_bloco_institucional(texto): #remover bloco institucional
    match = re.search(r'(?i)\n\s*(1[\.\s]|INTRODUÇÃO|APRESENTAÇÃO)\b', texto)
    if match:
        return texto[match.start():]
    return texto

def limpar_texto_avancado(texto):
    for char in CARACTERES_CORROMPIDOS:
        texto = texto.replace(char, '')

    for padrao in PADROES_TRECHOS_REMOVER:
        texto = re.sub(padrao, '', texto, flags=re.DOTALL | re.IGNORECASE)

    linhas = texto.splitlines()
    linhas_filtradas = []
    for linha in linhas:
        if any(re.match(p, linha.strip(), re.IGNORECASE) for p in PADROES_LINHAS_REMOVER):
            continue
        if len(linha.strip()) < 3:
            continue
        linhas_filtradas.append(linha)

    texto_base = "\n".join(linhas_filtradas)
    texto_sem_sumario = remover_blocos_de_sumario(texto_base)
    texto_sem_cabecalho = remover_bloco_institucional(texto_sem_sumario)
    texto_final = re.sub(r'\n{2,}', '\n\n', texto_sem_cabecalho)

    return texto_final.strip()

# === EXECUÇÃO ===

print(f"🔍 Limpando arquivos em: {PASTA_ORIGEM}")
for arquivo in os.listdir(PASTA_ORIGEM):
    if arquivo.endswith(".txt"):
        origem = os.path.join(PASTA_ORIGEM, arquivo)
        destino = os.path.join(PASTA_DESTINO, arquivo)

        with open(origem, "r", encoding="utf-8", errors="ignore") as f:
            conteudo = f.read()

        texto_limpo = limpar_texto_avancado(conteudo)

        with open(destino, "w", encoding="utf-8") as f:
            f.write(texto_limpo)

        print(f"✅ Arquivo limpo e salvo: {arquivo}")

print("🏁 Limpeza avançada com sumário, ficha institucional e cabeçalho finalizada.")


🔍 Limpando arquivos em: C:\Users\isisi\Documents\IAVS_PROJETO\textos_convertidos
✅ Arquivo limpo e salvo: adenovírus_guia_1.txt
✅ Arquivo limpo e salvo: arboviroses_nota_técnica_1.txt
✅ Arquivo limpo e salvo: cancro_mole_protocolo_1.txt
✅ Arquivo limpo e salvo: chikungunya_nota_técnica_1.txt
✅ Arquivo limpo e salvo: chikungunya_protocolo_1.txt
✅ Arquivo limpo e salvo: chikungunya_protocolo_2.txt
✅ Arquivo limpo e salvo: chikungunya_protocolo_3.txt
✅ Arquivo limpo e salvo: clamídia_protocolo_1.txt
✅ Arquivo limpo e salvo: coinfecção_com_hepatite_b_nota_técnica_1.txt
✅ Arquivo limpo e salvo: coqueluche_nota_técnica_1.txt
✅ Arquivo limpo e salvo: coqueluche_nota_técnica_2.txt
✅ Arquivo limpo e salvo: coqueluche_nota_técnica_3.txt
✅ Arquivo limpo e salvo: covid-19_guia_1.txt
✅ Arquivo limpo e salvo: covid-19_nota_técnica_1.txt
✅ Arquivo limpo e salvo: cólera_nota_técnica_1.txt
✅ Arquivo limpo e salvo: dengue_nota_técnica_1.txt
✅ Arquivo limpo e salvo: dengue_protocolo_1.txt
✅ Arquivo limpo

In [8]:
# === CONFIGURAÇÕES ===
# Pasta onde os arquivos limpos estão armazenados
PASTA_LIMPOS = r"C:\Users\isisi\Documents\IAVS_PROJETO\textos_limpos" # Pasta de destino para os arquivos limpos
# Caminho do log de saída
LOG_SAIDA = r"C:\Users\isisi\Documents\IAVS_PROJETO\log_qualidade_limpos.csv" # Garante que a pasta de destino existe

dados = [] # Cria uma lista para armazenar os dados dos arquivos

for arquivo in os.listdir(PASTA_LIMPOS):#para cada arquivo na pasta de arquivos limpos
    # Verifica se o arquivo é um .txt
    if arquivo.endswith(".txt"):# se o arquivo termina com .txt
        # Define o caminho completo do arquivo
        caminho = os.path.join(PASTA_LIMPOS, arquivo)#concatena o caminho da pasta com o nome do arquivo

        # Lê o conteúdo do arquivo
        with open(caminho, "r", encoding="utf-8", errors="ignore") as f:# abre o arquivo para leitura
            # Lê o conteúdo do arquivo
            conteudo = f.read()

        n_linhas = len(conteudo.strip().splitlines())# conta o número de linhas no conteúdo
        # Conta o número de palavras e o tamanho total do conteúdo
        # Usa split() para contar palavras, que separa por espaços em branco
        # e len() para contar o número total de caracteres
        n_palavras = len(conteudo.split())
        tamanho = len(conteudo)

        status = "✅ OK"
        if tamanho < 500 or n_linhas < 5:
            status = "⚠️ Suspeito (vazio ou ruidoso)"# se o tamanho do conteúdo for menor que 500 caracteres ou o número de linhas for menor que 5, marca como suspeito

        # Adiciona os dados do arquivo à lista

        dados.append({
            "Arquivo": arquivo,
            "Linhas": n_linhas,
            "Palavras": n_palavras,
            "Caracteres": tamanho,
            "Status": status
        })

# Salva log como CSV
df = pd.DataFrame(dados)
df.to_csv(LOG_SAIDA, index=False, encoding="utf-8")

print(f"📄 Log de qualidade salvo em: {LOG_SAIDA}")


📄 Log de qualidade salvo em: C:\Users\isisi\Documents\IAVS_PROJETO\log_qualidade_limpos.csv


## Etapa 6 – Verificação dos Arquivos Limpos
Esta etapa realiza uma verificação simples e rápida para conferir se os arquivos .txt limpos foram gerados corretamente e estão disponíveis na pasta de destino (textos_limpos).

**Objetivos:**
- Listar todos os arquivos presentes na pasta textos_limpos;

- Garantir visualmente que o processo de limpeza foi concluído com sucesso para cada documento convertido.

*Essa etapa funciona como um checkpoint antes de seguir para a organização final por doença e tema. Ela ajuda a identificar rapidamente eventuais falhas ou ausências de arquivos.*

In [9]:
# Exibe no terminal os arquivos que foram salvos após a limpeza
print("Arquivos encontrados em PASTA_LIMPOS:")

# Lista o conteúdo da pasta de textos limpos
print(os.listdir(PASTA_LIMPOS))

Arquivos encontrados em PASTA_LIMPOS:
['adenovírus_guia_1.txt', 'arboviroses_nota_técnica_1.txt', 'cancro_mole_protocolo_1.txt', 'chikungunya_nota_técnica_1.txt', 'chikungunya_protocolo_1.txt', 'chikungunya_protocolo_2.txt', 'chikungunya_protocolo_3.txt', 'clamídia_protocolo_1.txt', 'coinfecção_com_hepatite_b_nota_técnica_1.txt', 'coqueluche_nota_técnica_1.txt', 'coqueluche_nota_técnica_2.txt', 'coqueluche_nota_técnica_3.txt', 'covid-19_guia_1.txt', 'covid-19_nota_técnica_1.txt', 'cólera_nota_técnica_1.txt', 'dengue_nota_técnica_1.txt', 'dengue_protocolo_1.txt', 'dengue_protocolo_2.txt', 'dengue_protocolo_3.txt', 'dengue_protocolo_4.txt', 'difteria_nota_técnica_1.txt', 'entre_outras_doenças_imunopreveníveis_nota_técnica_1.txt', 'entre_outras_protocolo_1.txt', 'entre_outras_protocolo_2.txt', 'febre_amarela_nota_técnica_1.txt', 'febre_amarela_protocolo_1.txt', 'febre_amarela_protocolo_2.txt', 'febre_amarela_protocolo_3.txt', 'febre_do_nilo_ocidental_nota_técnica_1.txt', 'gonorreia_protoc